# Task 6.2 from *SemEval*
## Imports

In [1]:
import torch

from src.dataprocessing import preprocesser, postprocesser
from src.model import model

import json

## Input Data Preprocessing

In [2]:
train_file_path = "SEMEVAL-2021-task6-corpus/data/training_set_task2.txt"
with open(train_file_path, "r") as file:
    train_data = json.load(file)

dev_file_path = "SEMEVAL-2021-task6-corpus/data/dev_set_task2.txt"
with open(dev_file_path, "r") as file:
    dev_data = json.load(file)

test_file_path = "SEMEVAL-2021-task6-corpus/data/test_set_task2.txt"
with open(dev_file_path, "r") as file:
    test_data = json.load(file)

input_tdata, output_tdata = preprocesser.preprocess_data(train_data)
input_vdata, output_vdata = preprocesser.preprocess_data(dev_data)

In [3]:
model.train(input_tdata, output_tdata, input_vdata, output_vdata)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able

## Results & Post-Processing

In [3]:
model = torch.load('models/MEMbErt.pth')
predictions = []

with torch.no_grad():
    for meme in input_vdata:
        model.eval()
        predictions.append(model(' '.join(meme)).cpu().numpy())

In [4]:
output = postprocesser.postprocess_data(dev_data, input_vdata, predictions, model.tokenizer, threshold=0.2)
with open("out/out.txt", "w") as f:
    json.dump(output, f)

## Evaluation
For evaluation, type:

`python3 SEMEVAL-2021-task6-corpus/scorer/task2/task-2-semeval21_scorer.py -s out/out.txt -r SEMEVAL-2021-task6-corpus/data/dev_set_task2.txt -p SEMEVAL-2021-task6-corpus/techniques_list_task1-2.txt
`